In [3]:
from paddlenlp import Taskflow
from xpinyin import Pinyin
tagger = Taskflow("pos_tagging")
corrector = Taskflow("text_correction") # ernie


c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-04-25 09:24:55,456] [    INFO] - Already cached C:\Users\Administrator\.paddlenlp\models\ernie-1.0\vocab.txt
[2023-04-25 09:24:55,471] [    INFO] - tokenizer config file saved in C:\Users\Administrator\.paddlenlp\models\ernie-1.0\tokenizer_config.json
[2023-04-25 09:24:55,473] [    INFO] - Special tokens file saved in C:\Users\Administrator\.paddlenlp\models\ernie-1.0\special_tokens_map.json


In [4]:
def getEditDistance(s, t):
    def Minimum(a, b, c):
        im = a if a < b else b
        return im if im < c else c
    p = Pinyin()
    s = p.get_pinyin(s, ',', tone_marks='numbers')
    t = p.get_pinyin(t, ',', tone_marks='numbers')
    n = len(s)
    m = len(t)
    if n == 0 :
        return m
    if m == 0 :
        return n
    d = [([0] * (m + 1)) for i in range(n + 1)]
    for i in range(0, n + 1):
        d[i][0] = i
    for j in range(0, m + 1):
        d[0][j] = j
    for i in range(1, n + 1):
        s_i = s[i -1]
        for j in range(1, m + 1):
            t_j = t[j -1]
            cost = 0 if s_i == t_j else 1
            d[i][j] = Minimum(d[i - 1][j] + 1, d[i][j - 1] + 1,
                d[i - 1][j - 1] + cost)
    # print(d)
    return d[n][m]


getEditDistance('三','山')

1

In [5]:
texts =  [
        '真麻烦你了。希望你们好好的跳舞',
        '少先队员因该为老人让坐',
        '机七学习是人工智能领遇最能体现智能的一个分知',
        '一只小鱼船浮在平净的河面上',
        '我的家乡是有明的渔米之乡',
    ]

In [6]:
def get_corrected(texts):
    ls = corrector(texts)
    print(ls)
    corrected_texts = []
    positions =[]
    for i in ls:
        corrected_texts += [i['target']]
        positions += [i['errors']]    
    return corrected_texts , positions

In [7]:
corrected,positons = get_corrected(texts)

[{'source': '真麻烦你了。希望你们好好的跳舞', 'target': '真麻烦你了。希望你们好好地跳舞', 'errors': [{'position': 12, 'correction': {'的': '地'}}]}, {'source': '少先队员因该为老人让坐', 'target': '少先队员应该为老人让座', 'errors': [{'position': 4, 'correction': {'因': '应'}}, {'position': 10, 'correction': {'坐': '座'}}]}, {'source': '机七学习是人工智能领遇最能体现智能的一个分知', 'target': '机器学习是人工智能领域最能体现智能的一个分之', 'errors': [{'position': 1, 'correction': {'七': '器'}}, {'position': 10, 'correction': {'遇': '域'}}, {'position': 21, 'correction': {'知': '之'}}]}, {'source': '一只小鱼船浮在平净的河面上', 'target': '一只小鱼船浮在平净的河面上', 'errors': []}, {'source': '我的家乡是有明的渔米之乡', 'target': '我的家乡是有名的渔米之乡', 'errors': [{'position': 6, 'correction': {'明': '名'}}]}]


In [8]:
def get_tag_changes(correction):
    res = [{'source':correction[0]['source']}]
    if correction[0]['errors'] == []:
        return res , 'No errors detected'
    else:
        source_tags = tagger(correction[0]['source'])
        target_tags = tagger(correction[0]['target'])
        for i in correction[0]['errors']:
            t = list(i['correction'].keys())[0]
            s = list(i['correction'].values())[0]

            sentence_length = 0
            for j in source_tags:
                sentence_length += len(j[0])
                if sentence_length >= i['position']+1:
                    break
                
            sentence_length = 0
            for k in target_tags:
                sentence_length += len(k[0])
                if sentence_length >= i['position']+1:
                    break
            
            # check if typo
            typo = False
            for char in k[0]:
                if getEditDistance(j[0],char) <=2 :
                    typo = True

            if typo == True:
                resdict = {'position':i['position'],'tag_source':k[1],'target_tag':k[1],'target_word':k[0]}
            else:
                resdict = {'position':i['position'],'tag_source':j[1],'target_tag':k[1],'target_word':k[0]}
            
            res.append(resdict)


        return res




In [9]:
texts = [
    "我睡觉得很多，没有意思。",
    "我跟朋友说，我想看看那边，到底真的有同性人吗？",
    "然后，我们要去下一个节目，但是所有的活动都要排队，而且等候时间差不多100分",
    "我喜欢它，因为它是简单的和安静。",
    "因为她会说一点儿英语和日语，所以她英语和日语的歌曲，当然还中文唱歌。",
    "如果我不能那些问题获得好分数的话，考试合格很难。"
]

In [13]:
def helper_ernie(texts):
    return [i['target'] for i in  corrector(texts)]

In [17]:
p = helper_ernie

In [18]:
p(texts)

['我睡觉得很多，没有意思。',
 '我跟朋友说，我想看看那边，到底真的有同性人吗？',
 '然后，我们要去下一个节目，但是所有的活动都要排队，而且等候时间差不多100分',
 '我喜欢它，因为它是简单的和安静。',
 '因为她会说一点儿英语和日语，所以她英语和日语的歌曲，当然还中文唱歌。',
 '如果我不能哪些问题获得好分数的话，考试合格很难。']

In [11]:
for i in texts:
    correction =  corrector(i)
    print(get_tag_changes(correction)[1])

No errors detected
No errors detected
No errors detected
No errors detected
No errors detected
{'position': 5, 'tag_source': 'r', 'target_tag': 'r', 'target_word': '哪些'}
